In [1]:
# Tokens for connection to different services
from clan_tools.secrets.Vault import Vault
Vault().get_secrets()

# Connector to YT by ClickHouse
from clan_tools.data_adapters.ClickHouseYTAdapter import ClickHouseYTAdapter
ch_adapter = ClickHouseYTAdapter()

# Connector to YT
from clan_tools.data_adapters.YTAdapter import YTAdapter
yt_adapter = YTAdapter()

import numpy as np
import itertools as itert
import pandas as pd

In [2]:
dm_yc_cons_path = '\"//home/cloud-dwh/data/prod/cdm/dm_yc_consumption\"'

In [3]:
service_name_sql = '''
    select
        sku_service_group as service_group
    from
        {table_path}
    group by
        sku_service_group
'''.format(table_path=dm_yc_cons_path)

service_group_arr = ch_adapter.execute_query(query=service_name_sql, to_pandas=True).service_group.values
service_group_arr = service_group_arr[service_group_arr != None]

In [4]:
service_group_arr

array(['Kubernetes', 'Support', 'Professional Services',
       'Business & Dev Tools', 'ML and AI', 'Serverless', 'Marketplace',
       'Adjustments', 'Data Storage and Analytics', 'Infrastructure'],
      dtype=object)

In [5]:
service_groups_combanations = []

for size in range(1, len(service_group_arr) + 1):
    service_groups_combanations = \
        service_groups_combanations + \
        [
            (', '.join(combination), len(combination))
            for combination in itert.combinations(service_group_arr, size)
        ]

df_service_groups_combanations = pd.DataFrame(data=service_groups_combanations, columns=['service_groups_combanations', 'service_groups_num'])

In [6]:
df_service_groups_combanations


,service_groups_combanations,service_groups_num
0,Kubernetes,1
1,Support,1
2,Professional Services,1
3,Business & Dev Tools,1
4,ML and AI,1
...,...,...
1018,"Kubernetes, Support, Professional Services, ML...",9
1019,"Kubernetes, Support, Business & Dev Tools, ML ...",9
1020,"Kubernetes, Professional Services, Business & ...",9
1021,"Support, Professional Services, Business & Dev...",9


In [8]:
df_schema = [
    {"name": "service_groups_combanations", "type": "string"},
    {"name": "service_groups_num", "type": "int64"}
]

yt_adapter.save_result(
    result_path='//home/cloud_analytics/data_swamp/projects/service_groups_penetration/service_group_combinations',
    df=df_service_groups_combanations,
    schema=df_schema,
    append=False
)